In [1]:
from langchain_community.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L12-v2")

C:\Users\fzehr\AppData\Local\Temp\ipykernel_75284\3565532808.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L12-v2")
d:\RAG_e-commerce\RAG_e-commerce\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
print(embeddings)

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='all-MiniLM-L12-v2' cache_folder=None model_kwargs={} encode_kwargs={} multi_process=False show_progress=False


In [12]:
from connect_db import create_connection
from collections import defaultdict


In [36]:
query_result = embeddings.embed_query("heelo world")
len(query_result)

384

In [13]:
from langchain_core.documents import Document


In [1]:
import os
import time
import dotenv
import os

from pinecone import Pinecone, ServerlessSpec
apikey= os.getenv("PINECONE_API_KEY")
pc= Pinecone(api_key = apikey)




d:\RAG_e-commerce\RAG_e-commerce\venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [5]:
index = pc.Index("ecommerce-2")

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L12-v2")

In [6]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [14]:
def fetch_data_from_postgres():
    connection = create_connection()
    cursor = connection.cursor()
    query = """
        -- her sipariş kullanıcı ve ürün bilgileir ile birleşir
SELECT
    u.user_id,
    u.user_name AS user_name,
    o.order_id,
    o.product_id,
    o.order_date,
    p.product_name AS product_name,
    p.category AS product_category
FROM
    orders o
JOIN
    users u ON o.user_id = u.user_id
JOIN
    products p ON o.product_id = p.product_id
ORDER BY 
    o.order_date DESC;
        """
    cursor.execute(query)
    data = cursor.fetchall()
    print("veri alınıd")
    connection.close()
    return data



In [26]:
def prepare_docs(data):
    documents = []

    grouped_orders = defaultdict(list)
    for row in data:
        user_id, user_name, order_id, product_id, order_date, product_name, product_category = row

        order_date_trimmed = order_date.replace(microsecond=0)

        key = (user_id, order_date_trimmed)

        if key not in grouped_orders:
            grouped_orders[key] = {
                'user_name': user_name,  # Kullanıcı adı ekleniyor
                'products': []          # Ürünler için boş liste
            }

    # Ürün bilgisi ekleniyor
        grouped_orders[key]['products'].append({
            'product_name': product_name,
            'category': product_category
        })

    print(f"groped orders into {len(grouped_orders)} unique user_date combinations")

    for (user_id, order_date), details in grouped_orders.items():
        user_name = details['user_name']
        products = details['products']
        
        # Ürün bilgilerini birleştir, sadece dict olanları al
        products_text = " and ".join([
            f"{p['product_name']} ({p['category']})"
            for p in products if isinstance(p, dict)
        ])

    
        text = f"User {user_name} ordered {products_text} on {order_date}"
        #print(text)
        # embedding = embeddings.embed_documents(text)
        
        # Metadata hazırla
        metadata = {
            "user_id": user_id,
            "user_name": user_name,
            "order_date": str(order_date),
            "products": [p['product_name'] for p in products if isinstance(p, dict)],
            "categories": [p['category'] for p in products if isinstance(p, dict)]
        }
        # print(metadata)

        doc = Document(
            page_content = text,
            metadata=metadata
        )
        documents.append(doc)

        # unique_id = f"{user_id}_{order_date}"
        # embedding_list.append((unique_id, embedding, metadata))
        
        # embeddings.append((user_id, embedding, metadata))

    return documents

In [27]:
data = fetch_data_from_postgres()
docs = prepare_docs(data)

connection succes
veri alınıd
groped orders into 140 unique user_date combinations


In [31]:
docs[1]

Document(metadata={'user_id': 346, 'user_name': 'Jacob Fischer', 'order_date': '2025-01-01 14:16:55', 'products': ['Smartwatch (Pink)', 'Plus Size Coat (Green)'], 'categories': ['Electronics', 'Clothing']}, page_content='User Jacob Fischer ordered Smartwatch (Pink) (Electronics) and Plus Size Coat (Green) (Clothing) on 2025-01-01 14:16:55')

In [33]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(docs))]


In [34]:
vector_store.add_documents(documents=docs, ids = uuids)

['f386a5c8-219b-41ee-b120-49a805375946',
 'ccb54243-89d8-4862-a85a-bc3e63974c17',
 'bb9243ac-09a8-482b-a5c2-cafbab09f5ef',
 '4db47027-bf41-485d-bb79-b7819a777e77',
 'd0011354-9351-4298-9625-8f8b8eef44d3',
 'ecf8a6b5-1d60-450a-99e9-28dd586f0a23',
 '49c170e0-e1b5-4f04-a1d9-b17e46cc0106',
 'bdfbc029-9f15-43c5-ac0d-a86ffbfea663',
 '13e6a811-9fad-4eab-9afc-105e6b9088b2',
 'c91bef84-3673-412c-ac57-200263c81d34',
 'be4315b2-3603-4c3d-a239-14056de30f64',
 '18356b7e-e8ee-4796-931e-3d518be6bb4d',
 '27d06b6e-c10f-44e4-8bc5-6e7362dfeea7',
 'fb9ec26b-c798-4468-9741-947730f6ddab',
 'a4196655-ede5-4e7c-a7fa-38cdb2851e4f',
 '29ca8f0a-b7bc-4efe-b5fd-8e21e0128629',
 '99bb6198-be4c-4199-a9d6-c879fe73cfdb',
 'a101f0f3-42fe-430d-b373-266adce1688c',
 '3186bf54-9bc2-4117-9a38-874f4961c201',
 '0318c63d-cb8c-4bb4-9800-811adbc8e6b9',
 'b61121fd-d07f-477a-b542-4a05bd858013',
 'e69f40f6-768e-4a63-8905-7fb927039175',
 'c68b9c1a-f563-4326-a372-94dad2e2f0e0',
 'da094ece-3807-4f40-aa1c-30d770f13f2d',
 '9bc0eb81-80eb-

In [38]:
results = vector_store.similarity_search_with_score(
    "who bus a smartphone",
    k=3,
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.490285] User Cheryl Fuller ordered Smartphone (Yellow) (Electronics) on 2025-01-01 14:11:59 [{'categories': ['Electronics'], 'order_date': '2025-01-01 14:11:59', 'products': ['Smartphone (Yellow)'], 'user_id': 350.0, 'user_name': 'Cheryl Fuller'}]
* [SIM=0.379550] User Marcus Williams ordered Powerbank (Purple) (Electronics) and Smartphone (Red) (Electronics) on 2025-01-01 13:06:39 [{'categories': ['Electronics', 'Electronics'], 'order_date': '2025-01-01 13:06:39', 'products': ['Powerbank (Purple)', 'Smartphone (Red)'], 'user_id': 349.0, 'user_name': 'Marcus Williams'}]
* [SIM=0.373453] User Mr. Joseph Taylor ordered Smartphone (Blue) (Electronics) and Food Processor (Purple) (Home Goods) on 2025-01-01 14:12:56 [{'categories': ['Electronics', 'Home Goods'], 'order_date': '2025-01-01 14:12:56', 'products': ['Smartphone (Blue)', 'Food Processor (Purple)'], 'user_id': 398.0, 'user_name': 'Mr. Joseph Taylor'}]


In [44]:
def retrieve_query(query, k=100):
    matching_docs = vector_store.similarity_search(query, k=k)
    return matching_docs

In [41]:
from langchain.chains.question_answering import load_qa_chain
from langchain_community.chat_models import ChatOpenAI

In [42]:
llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0)

C:\Users\fzehr\AppData\Local\Temp\ipykernel_51260\1360019337.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0)


In [45]:
chain = load_qa_chain(llm,chain_type="stuff")

C:\Users\fzehr\AppData\Local\Temp\ipykernel_51260\3203274189.py:1: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain = load_qa_chain(llm,chain_type="stuff")


In [43]:
from langchain.prompts import ChatPromptTemplate

rag_template = """Answer the following question based on this context:
{context}
Question: {question}
"""
rag_prompt = ChatPromptTemplate.from_template(rag_template)

In [47]:
def retrieve_answer(query):
    doc_search = retrieve_query(query)
    print(doc_search)
    response = chain.run(input_documents = doc_search, question = query )
    return response

In [49]:
query = "what time the order many"
answer = retrieve_answer(query)
print(answer)

[Document(id='5322198c-4756-4804-8a00-187ea9703917', metadata={'categories': ['Home Goods', 'Home Goods'], 'order_date': '2025-01-01 14:15:03', 'products': ['Table Clock (Orange)', 'Table Clock (White)'], 'user_id': 494.0, 'user_name': 'Andrea Thomas'}, page_content='User Andrea Thomas ordered Table Clock (Orange) (Home Goods) and Table Clock (White) (Home Goods) on 2025-01-01 14:15:03'), Document(id='d17b5870-0393-4fde-8b3f-af88238cfe76', metadata={'categories': ['Home Goods', 'Personal Care', 'Flower Type'], 'order_date': '2024-12-30 10:24:46', 'products': ['Table Clock (Black)', 'Perfume (Yellow)', 'Succulent (Brown)'], 'user_id': 459.0, 'user_name': 'Daniel Taylor'}, page_content='User Daniel Taylor ordered Table Clock (Black) (Home Goods) and Perfume (Yellow) (Personal Care) and Succulent (Brown) (Flower Type) on 2024-12-30 10:24:46'), Document(id='1a33c04c-faa2-4d7f-ba9a-74482f15477c', metadata={'categories': ['Home Goods', 'Flower Type', 'Flower Type', 'Home Goods'], 'order_date